In [2]:
import pandas as pd
import numpy as np

In [3]:
titanic= pd.read_csv('data/titanic.csv')
titanic.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [4]:
def process_titanic(dataframe):
    # Create a copy of the dataframe to avoid modifying the original
    
    df = dataframe.copy()

    # Step 1: Feature engineering
    df['family_size'] = df['SibSp'] + df['Parch'] + 1  # Total family size
    
    df['is_alone'] = (df['family_size'] == 1).astype(int)  # Whether the passenger is alone
    
    df['has_cabin'] = df['Cabin'].notnull().astype(int)  # Whether the passenger has cabin information

    
    # Step 2: Encoding categorical features
    
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})  # Encode Sex as 0 and 1
    
    df['Embarked'] = df['Embarked'].fillna('Unknown').astype('category').cat.codes  # Encode Embarked
    
    df['Name'] = df['Name'].astype('category').cat.codes  # Encode Name

    # Step 3: Processing ticket column
    
    df['Ticket'] = df['Ticket'].str.split(' ').str[-1]  # Extract the last part of the Ticket

    # Step 4: Handling missing values
    
    df['Age_missing'] = df['Age'].isnull().astype(int)  # Add a column to indicate missing Age
    
    df['Age'] = df['Age'].fillna(df['Age'].mean()).astype(int)  # Fill missing Age with mean and convert to int

    # Step 5: Removing unwanted records
    
    df = df[df['Ticket'] != 'LINE']  # Drop rows where Ticket equals 'LINE'

    # Return the processed dataframe (X) and target column (y)
    return df.drop(['Cabin', 'Survived'], axis=1), df['Survived']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x, y = process_titanic(titanic)
display(x.head(), y.head())
x.drop(columns = 'Name', inplace=True)
print(x.shape , y.shape)
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2)

PassengerId  Pclass  Name  Sex  Age  SibSp  Parch   Ticket     Fare  \
0            1       3   108    0   22      1      0    21171   7.2500   
1            2       1   190    1   38      1      0    17599  71.2833   
2            3       3   353    1   26      0      0  3101282   7.9250   
3            4       1   272    1   35      1      0   113803  53.1000   
4            5       3    15    0   35      0      0   373450   8.0500   

   Embarked  family_size  is_alone  has_cabin  Age_missing  
0         2            2         0          0            0  
1         0            2         0          1            0  
2         2            1         1          0            0  
3         2            2         0          1            0  
4         2            1         1          0            0

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

(887, 13) (887,)


In [6]:
clf = RandomForestClassifier(100)
clf.fit(X_train, y_train) 

RandomForestClassifier()

In [7]:
clf.score(X_train, y_train), clf.score(X_test, y_test) # all data

(1.0, 0.8314606741573034)